In [1]:
import numpy as np
import torch
import math
import pytorchvis

In [2]:
class SocialSig(torch.nn.Module):
    '''
    Class to create the social signature image
    '''
    def __init__(self, X, W):
        super(SocialSig, self).__init__()
        self.W = W
        self.outDim = [10,10]
        self.inDim = math.ceil(math.sqrt(len(X)))
        self.X = X
        

    def forward(self):
        xTemp = torch.stack([self.X, self.W])
        X = torch.sort(xTemp, dim=1, descending=False)
        buildImage = torch.reshape(X[0][0],(1, 1, self.inDim, self.inDim))
        return torch.nn.functional.interpolate(buildImage, size=([self.outDim[0], self.outDim[1]]), mode='bilinear')    

In [3]:
class SocialSigNet(torch.nn.Module):
    def __init__(self, X):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.W = torch.nn.Parameter(torch.tensor(np.arange(0,len(X)), dtype = torch.float32, requires_grad=True))
        self.SocialSig = SocialSig(X,self.W)                
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(10,10))  #10,10 is static here.  Will need to be dynamic based on user dim settings.

    def forward(self, X):
        print("    W at beginning: ", self.W.data)
        out = self.SocialSig()
        out = self.maxPool(out)
        
        return out

In [4]:
# Create random tensors to hold input and outputs.
X = torch.tensor([-1.6848, -0.6, -93.2, -182.2], requires_grad = True)
y = torch.tensor([0.0359], requires_grad = True)

# Construct our model by instantiating the class defined above
model = SocialSigNet(X=X)
#print(dir(model))
print(list(model.parameters()))

[Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)]


In [5]:
# from torchviz import make_dot

# make_dot(model(X), params=dict(list(model.named_parameters()))).render("backPropViz", format="png")

In [6]:
# Setting the LR to something dumb just to test that things are indeed working
lr = 10
criterion = torch.nn.MSELoss(reduction='none')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)


def update_function(param, grad, loss, learning_rate):
    return param - learning_rate * grad


for t in range(5):
    print("EPOCH: ", t)
    y_pred = model(X)
    print("    Predicted Y:    ", y_pred)
    loss = criterion(y_pred, y)
    print("    Loss:           ", loss)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    grad = torch.autograd.grad(outputs=loss, inputs=X, retain_graph = True)
    print("    Gradient:       ", grad[0])
    loss.backward()
    optimizer.step()
    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
    with torch.no_grad():
        for p in model.parameters():
            print("    In with:        ", p.data)
            new_val = update_function(p, grad[0], loss, lr)
            p.copy_(new_val)
    
    print("\n")

print("Model.parameters")
print(model.parameters)
print("SSParam")
print(list(model.SocialSig.parameters()))
print("is_leaf")
print(list(model.SocialSig.parameters())[0].is_leaf)
print("gradfn")
print(list(model.SocialSig.parameters())[0].grad_fn)
print("Grad")
print(list(model.SocialSig.parameters())[0].grad)

EPOCH:  0
    W at beginning:  tensor([0., 1., 2., 3.])
    Predicted Y:     tensor([[[[-0.6000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
    Loss:            tensor([[[[0.4044]]]], grad_fn=<MseLossBackward>)
    Gradient:        tensor([ 0.0000, -1.2718,  0.0000,  0.0000])
    In with:         tensor([0., 1., 2., 3.])


EPOCH:  1
    W at beginning:  tensor([ 0.0000, 13.7180,  2.0000,  3.0000])
    Predicted Y:     tensor([[[[-0.6000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
    Loss:            tensor([[[[0.4044]]]], grad_fn=<MseLossBackward>)
    Gradient:        tensor([ 0.0000, -1.2718,  0.0000,  0.0000])
    In with:         tensor([ 0.0000, 13.7180,  2.0000,  3.0000])


EPOCH:  2
    W at beginning:  tensor([ 0.0000, 26.4360,  2.0000,  3.0000])
    Predicted Y:     tensor([[[[-0.6000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
    Loss:            tensor([[[[0.4044]]]], grad_fn=<MseLossBackward>)
    Gradient:        tensor([ 0.0000, -1.2718,  0.0000,  0.0000])
    In wit

/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
